In [1]:
# %% [code]
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import os
import json
from typing import Dict, Optional, Any
import sys
from langdetect import detect

# MongoDBInserter가 정의된 공식 문서를 기반으로 가져옵니다.
# (MongoDBInserter는 src.load_data에 정의되어 있다고 가정)
from src.load_data import MongoDBInserter

# 주피터 노트북에서는 __file__이 정의되어 있지 않으므로 현재 작업 디렉토리를 사용합니다.
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
class BookInfoCrawler:
    def __init__(self, num_reviews: int, webdriver_path: Optional[str] = None):
        """_summary_
        책 정보와 리뷰를 수집하는 웹 크롤러 클래스입니다.
        Args:
            num_reviews (int): _description_
            webdriver_path (Optional[str], optional): _description_. Defaults to None.
        """
   
        self.webdriver_path = webdriver_path
        self.num_reviews = num_reviews
        self.driver = None
        self.book_info: Dict[str, Any] = {}
        self.output = None

        self.overlay_selector = "div.Overlay.Overlay--floating"
        self.login_overlay_dismiss_selector = (
            "body > div.Overlay.Overlay--floating > div > div.Overlay__header > div > div > button"
        )

        # 버튼 클릭 관련 CSS selector 정보
        self.book_details_button_selector = "button[aria-label='Book details and editions']"
        self.view_more_reviews_button_selector = "a.Button.Button--transparent.Button--medium"

        # 언어 필터 관련 XPath 정보 (apply_language_filter 메서드에서 사용)
        self.launguage_filter = {
            "filter": "//*[@id='ReviewsSection']/div[5]/div[2]/div[1]/div[2]/div/button",
            "en_label": "/html/body/div[3]/div/div[2]/span/div/div[6]/div[2]/label",
            "apply": "/html/body/div[3]/div/div[3]/div[2]/button",
            "text": "/html/body/div[3]/div/div[2]/span/div/div[6]/div[2]/label/text()"
        }

        # 책 정보 selector (scrap_book 메서드에서 사용)
        self.book_selector = {
            'book_name': "h1[data-testid='bookTitle']",
            'book_image': "img.ResponsiveImage",
            'author': "span.ContributorLink__name[data-testid='name']",
            'total_star': "div.RatingStatistics__rating",
            'description': "span.Formatted",
            # published: 텍스트에 ' by '가 포함된 요소를 찾습니다.
            'published': "//div[@data-testid='contentContainer' and contains(text(), ' by ')]",
            # ISBN: 텍스트에 'ISBN'이라는 단어가 포함된 요소를 찾습니다.
            'ISBN': "//div[@data-testid='contentContainer' and contains(., 'ISBN')]"
        }

        # 리뷰 selector
        self.review_selectors = {
            "reviewer": "a[href*='/user/show']",
            "star": "span.RatingStars",  # 별점 정보 (aria-label 사용)
            "date": "a[href*='review/show']",
            "review": "span.Formatted"
        }

        self.review_xpaths = {
            "reviewer": "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/div/div/section[2]/span[1]/div/a",
            "date":     "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/span/a",
            "rating":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/div/span",
            "review":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[2]/section/div/div[1]/span"
        }


        self.show_more_button_selector = "button.Button.Button--secondary.Button--medium"
        self.show_more_button_xpath = '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button'





    def click_button(self, button_selector):
        """_summary_

        지정된 CSS selector를 가진 버튼을 클릭하는 범용 함수입니다.
        
        동작 과정:
        1. button_selector에 해당하는 버튼이 클릭 가능한 상태가 될 때까지 최대 10초 대기합니다.
        2. "div.Overlay.Overlay--floating" 요소(오버레이)가 사라질 때까지 대기합니다.
        3. 버튼 클릭 후 2초 대기합니다.
        
        예외 발생 시:
        - 오류 메시지를 출력하고, 만약 해당 버튼이 'More reviews and ratings' 버튼이면 self.book_info['reviews']를 빈 리스트로 설정합니다.

        Args:
            button_selector (_type_): _description_


        
        :param button_selector: 클릭할 버튼의 CSS selector (예: self.book_details_button_selector)
        """

        try:
            # 버튼이 클릭 가능한 상태가 될 때까지 대기
            button = WebDriverWait(self.driver, 7).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, button_selector))
            )

            # 오버레이가 사라질 때까지 대기 (없으면 무시)
            try:
                WebDriverWait(self.driver, 10).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.Overlay.Overlay--floating"))
                )
            except Exception as e:
                print("버튼 클릭 대기 중 오버레이 관련 문제 발생(무시):", e)

            # 버튼 클릭 후 잠시 대기
            button.click()
            print("버튼 클릭 성공!")
            time.sleep(2)

        except Exception as e:
            print(f"버튼 클릭 중 오류 발생 ({button_selector}):", e)
            # 예를 들어, 'More reviews and ratings' 버튼의 selector인 경우 reviews를 빈 리스트로 설정
            if button_selector == self.view_more_reviews_button_selector:
                self.book_info['reviews'] = []


    def apply_language_filter(self):
        """_summary_
        리뷰 페이지에서 언어 필터를 적용합니다.
        
        """
        try:
            filter_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.launguage_filter['filter']))
            )
            filter_button.click()
            time.sleep(1)
            en_label = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "/html/body/div[3]/div/div[2]/span/div/div[6]/div[label[starts-with(normalize-space(text()), 'English')]]/label")
                )
            )
            en_label.click()
            time.sleep(1)
            apply_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.launguage_filter['apply']))
            )
            apply_button.click()
            print("언어 필터 적용 완료")
        except Exception as e:
            print("언어 필터 적용 중 오류 발생:", e)


    def extract_book(self):
        """페이지에서 책 정보를 추출합니다."""
        # 먼저 책 제목(book_name) 요소가 로드될 때까지 대기합니다.
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, self.book_selector['book_name']))
            )
        except Exception as e:
            print("책 정보 로딩 중 오류 발생:", e)
        
        # 각 필드에 대해 지정된 셀렉터를 사용하여 정보를 추출합니다.
        for key, selector in self.book_selector.items():
            try:
                # published와 ISBN은 XPath로 찾습니다.
                if key in ['published', 'ISBN']:
                    element = self.driver.find_element(By.XPATH, selector)
                else:
                    element = self.driver.find_element(By.CSS_SELECTOR, selector)
                
                if key == 'book_image':
                    self.book_info[key] = element.get_attribute("src")
                else:
                    self.book_info[key] = element.text.strip()
            except Exception as e:
                print(f"{key} 정보 추출 중 오류 발생: {e}")
                self.book_info[key] = None




        


    def scrap_book(self):
        """_summary_
        도서 상세 페이지에서 책 정보를 추출한 후, 리뷰 페이지로 이동하여 리뷰를 수집합니다.
        """
        # 1) 페이지 로딩 시간을 충분히 주기 위해 임시로 3초 정도 대기 (테스트 코드와 동일 흐름)
        time.sleep(3)

        # 책 정보 더보기 버튼 클릭
        self.click_button(self.book_details_button_selector)

        # 언어 필터 적용
        self.apply_language_filter()

        # book_info{}로 책 정보 추출 (book_name, book_image, author, total_star, published, description, ISBN)
        self.book_info = {}
        self.extract_book()

        # book_info{}로 장르 추출 (genre)
        self.extract_genre()

        # 책 정보가 모두 존재하는지 확인
        if any(value is None for value in self.book_info.values()):
            print(f"책 정보 중 일부가 누락되었습니다: {self.book_info["book_name"]}")
            return
        
        # 오버레이 해제 시도 (필요에 따라 유지하거나 제거 가능)
        self.check_and_dismiss_overlay(timeout=5)

        # More reviews and ratings 버튼 클릭
        self.click_button(self.view_more_reviews_button_selector)


        # book_info{}로 리뷰 추출 (reviews)
        reviews = self.scrap_reviews()
        self.book_info['reviews'] = reviews


        return self.book_info

    def load_container(self, current_index, consecutive_failures, max_failures):
        """
        주어진 인덱스를 기반으로 리뷰 컨테이너를 로드하려 시도합니다.
        컨테이너를 찾지 못하면 연속 실패 횟수를 증가시키고,
        실패 횟수가 max_failures 이상이면 "show more reviews" 버튼을 클릭해 추가 컨테이너를 로딩합니다.
        
        :param current_index: 현재 접근할 컨테이너의 인덱스 (1부터 시작)
        :param consecutive_failures: 현재까지의 연속 실패 횟수
        :param max_failures: 허용 가능한 최대 연속 실패 횟수
        :return: (container, updated_current_index, updated_consecutive_failures)
                container: 찾은 리뷰 컨테이너(WebElement) 또는 None
        """
        container_xpath = f"//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[{current_index}]"
        try:
            container = self.driver.find_element(By.XPATH, container_xpath)
            return container, current_index, consecutive_failures
        except Exception:
            consecutive_failures += 1
            if consecutive_failures >= max_failures:
                try:
                    self.click_button(self.show_more_button_selector)
                    time.sleep(2)
                    consecutive_failures = 0
                except Exception as e:
                    print("더 이상 리뷰 로딩 불가 또는 show more reviews 클릭 중 오류 발생:", e)
                    return None, current_index, consecutive_failures
            current_index += 1
            return None, current_index, consecutive_failures

    def scrap_reviews(self, target_review_count=100):
        """
        리뷰 컨테이너에서 리뷰 정보를 추출합니다.
        추가 리뷰 로딩이 필요하면 "show more reviews" 버튼을 클릭합니다.
        """
        reviews = []
        current_index = 1
        max_failures = 3
        consecutive_failures = 0

        while len(reviews) < target_review_count:
            # 1) 우선 div[current_index]가 존재하는지 확인
            container_xpath = f"//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[{current_index}]"
            try:
                self.driver.find_element(By.XPATH, container_xpath)
                # 찾혔다면 실패 카운트 0으로 초기화
                consecutive_failures = 0
            except Exception:
                # 못 찾았다면, 실패 횟수 증가
                consecutive_failures += 1
                # 연속 실패 횟수가 일정 이상이면 => 'show more reviews' 클릭 시도
                if consecutive_failures >= max_failures:
                    try:
                        show_more_button = WebDriverWait(self.driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                        )
                        show_more_button.click()
                        time.sleep(2)
                        consecutive_failures = 0
                    except Exception as e:
                        print("show more reviews 버튼 클릭 실패 또는 더 이상 버튼이 없음:", e)
                        break
                current_index += 1
                continue

            # 2) 실제 리뷰 데이터 추출
            review_data = {}
            extraction_success = True
            for key, xpath_template in self.review_xpaths.items():
                # XPATH에서 INDEX 부분을 현재 current_index로 치환
                xpath = xpath_template.replace("INDEX", str(current_index))
                try:
                    element = self.driver.find_element(By.XPATH, xpath)
                    if key == "rating":
                        # 별점은 aria-label 속성에 들어있다고 가정
                        review_data[key] = element.get_attribute("aria-label").strip()
                    else:
                        # 리뷰어, 날짜, 리뷰 본문 등은 text로
                        review_data[key] = element.text.strip()
                except Exception:
                    # 만약 하나라도 못 찾으면 '해당 리뷰는 유효하지 않다'고 보고, 루프 탈출
                    extraction_success = False
                    break
            
            # 3) 리뷰가 제대로 추출되었고, review(본문)가 비어 있지 않다면 저장
            if extraction_success:
                # 1) 리뷰 본문이 비어있지 않은지 확인
                review_text = review_data.get("review", "")
                # 2) 영어인지 판별 (check_if_english)
                if review_text and self.check_if_english(review_text):
                    reviews.append(review_data)
                    consecutive_failures = 0
                else:
                    consecutive_failures += 1
            else:
                consecutive_failures += 1

            # 4) 목표 리뷰 개수를 채웠으면 중단
            if len(reviews) >= target_review_count:
                break

            current_index += 1

            # 5) 연속 실패가 계속되면 -> 'show more reviews' 버튼 클릭 시도
            if consecutive_failures >= max_failures:
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    consecutive_failures = 0
                except Exception as e:
                    print("더 이상 리뷰 로딩 불가 또는 show more reviews 클릭 중 오류 발생:", e)
                    break
        
        return reviews
    def scrap_reviews2(self, target_review_count=90):
        """
        리뷰 컨테이너의 xpath를 인덱스(1씩 증가하는 div[INDEX] 형식)로 접근하여 리뷰 데이터를 추출합니다.
        리뷰 데이터가 없으면 그냥 건너뛰고, 컨테이너가 부족할 경우 "show more reviews" 버튼을 클릭해 추가 컨테이너를 로딩합니다.
        
        :param target_review_count: 수집할 목표 리뷰 개수 (기본값 100)
        :return: 수집된 리뷰 데이터 리스트
        """
        reviews = []
        current_index = 1  # xpath의 div[INDEX] 부분을 1부터 시작
        consecutive_failures = 0
        max_failures = 3

        while len(reviews) < target_review_count:

            # 리뷰 컨테이너 로드 & 리뷰 컨테이너가 부족하면 Show more reviews 버튼 클릭
            container, current_index, consecutive_failures = self.load_container(current_index, consecutive_failures, max_failures)
            if container is None:
                # load_container에서 더 이상 로드할 컨테이너가 없거나 버튼 클릭 실패 시 None을 반환함
                break

            # 리뷰 추출 시도 (리뷰가 없으면 extract_review 함수가 None을 반환)
            review_data = self.extract_review(container)
            if review_data:
                reviews.append(review_data)
                consecutive_failures = 0
            else:
                consecutive_failures += 1

            current_index += 1

            if consecutive_failures >= max_failures:
                print("연속 리뷰 추출 실패 횟수 초과, 리뷰 수집 종료")
                break

        return reviews
    
    def extract_review(self, container):
        """
        주어진 리뷰 컨테이너 내에서 리뷰 데이터를 추출합니다.
        
        :param container: 리뷰 컨테이너 WebElement
        :return: 추출 성공 시 리뷰 데이터 딕셔너리, 실패 시 None 반환
        """
        review_data = {}
        try:
            # reviewer: 모든 <a> 태그를 검색한 후 href에 "/user/show"가 포함된 요소를 선택합니다.
            reviewer_elements = container.find_elements(By.TAG_NAME, "a")
            reviewer_text = ""
            for elem in reviewer_elements:
                href = elem.get_attribute("href")
                if href and "/user/show" in href:
                    reviewer_text = elem.text.strip()
                    break
            review_data["reviewer"] = reviewer_text

            # 별점 정보: 기존 selector 사용 (aria-label 이용)
            star_element = container.find_element(By.CSS_SELECTOR, "span.RatingStars")
            review_data["star"] = star_element.get_attribute("aria-label").strip()

            # 날짜 정보: 기존 selector 사용
            date_element = container.find_element(By.CSS_SELECTOR, "a[href*='review/show']")
            review_data["date"] = date_element.text.strip()

            # 리뷰 텍스트 정보: 기존 selector 사용
            review_element = container.find_element(By.CSS_SELECTOR, "span.Formatted")
            review_data["review"] = review_element.text.strip()

        except Exception as e:
            print("리뷰 데이터 추출 중 오류 발생:", e)
            return None

        return review_data





    def extract_genre(self):
        """
        상위 컨테이너 내에서 장르 정보를 추출하여 self.book_info['genre']에 리스트 형태로 저장합니다.
        만약 'Genres'라는 불필요한 텍스트가 있다면 이를 제외합니다.
        """
        time.sleep(1)
        try:
            # 장르가 포함된 상위 컨테이너를 먼저 찾습니다.
            genre_container = self.driver.find_element(By.CSS_SELECTOR, "div[data-testid='genresList']")
            # 컨테이너 내부에서 모든 장르 텍스트를 가진 요소들을 찾습니다.
            genre_elements = genre_container.find_elements(By.CSS_SELECTOR, "span.Button__labelItem")
            # "Genres"와 같이 실제 장르가 아닌 텍스트를 필터링합니다.
            genres = [element.text.strip() for element in genre_elements 
                    if element.text.strip() and element.text.strip().lower() != "genres"]
            self.book_info['genre'] = genres
        except Exception as e:
            print("장르 정보 추출 중 오류 발생:", e)
            self.book_info['genre'] = []

    def check_and_dismiss_overlay(self, timeout=5):
        """
        로그인 오버레이가 있으면 해제합니다.
        """
        end_time = time.time() + timeout
        dismissed = False
        while time.time() < end_time:
            try:
                dismiss_button = self.driver.find_element(By.CSS_SELECTOR, self.login_overlay_dismiss_selector)
                if dismiss_button.is_displayed() and dismiss_button.is_enabled():
                    dismiss_button.click()
                    print("로그인 오버레이가 해제되었습니다.")
                    dismissed = True
                    break
            except Exception:
                break
            time.sleep(0.5)
        return dismissed

    def scrap_list(self, json_file: str, start_index: int, end_index: int, mongo_inserter):
        """
        JSON 파일에 저장된 책 링크 데이터를 이용하여 지정한 인덱스 범위 내의 책을 처리합니다.
        JSON 파일 구조 예시:
            {
                "1": ["영문 책 제목", "https://www.goodreads.com/book/show/12345-book-name"],
                "2": ["영문 책 제목", "https://www.goodreads.com/book/show/67890-book-name"],
                ...
            }
        Args:
            json_file (str): 책 링크 데이터가 저장된 JSON 파일 경로.
            start_index (int): 처리 시작 인덱스 (포함).
            end_index (int): 처리 종료 인덱스 (포함).
            mongo_inserter: DB 저장을 위한 객체.
        """
        # JSON 파일에서 책 링크 데이터를 로드합니다.
        with open(json_file, "r", encoding="utf-8") as f:
            book_data_dict = json.load(f)
        
        # 지정한 인덱스 범위 내의 각 책에 대해 처리합니다.
        for idx in range(start_index, end_index + 1):
            key = str(idx)
            if key in book_data_dict:
                # JSON 데이터는 [책 제목, url] 형태로 저장되어 있다고 가정합니다.
                book_title, url = book_data_dict[key]
                print(f"책 링크 처리 (Index {idx}): {url}")
                
                # URL 접속
                self.driver.get(url)
                time.sleep(3)
                
                # 오버레이가 있다면 해제
                self.check_and_dismiss_overlay(timeout=5)
                
                # 책 상세 정보를 스크랩합니다.
                book_details = self.scrap_book()
                
                # 스크랩한 데이터를 DB에 저장합니다.
                #if not book_details:
                #    continue
                #mongo_inserter.run_di(book_details)
                self.append_book_details_to_json(book_details, 'database/nonfiction_data.json')
            else:
                print(f"Index {idx} not found in JSON data.")


    def load_existing_data(self, file_path):
        if not os.path.exists(file_path):
            return []
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 만약 JSON 구조가 배열이 아닌 딕셔너리 등이면, 여기서 에러 또는 형태 맞춤 필요
                if isinstance(data, list):
                    return data
                else:
                    # 예: 파일 구조가 list가 아니라면, 새로 list 초기화
                    return []
        except Exception as e:
            print("기존 데이터 로드 실패:", e)
            return []

    # 2) 새로운 book_details를 파일에 추가(append)하는 함수
    def append_book_details_to_json(self, book_details, file_path):
        # (A) 기존 데이터를 읽어온다
        existing_data = self.load_existing_data(file_path)
        
        # (B) 새 데이터를 list에 추가
        existing_data.append(book_details)
        
        # (C) 리스트 전체를 다시 파일에 저장 (덮어쓰기)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=4)

    def close_browser(self):
        """브라우저를 종료합니다."""
        if self.driver:
            self.driver.quit()

    def start_browser(self, webdriver_path: Optional[str] = None) -> webdriver.Chrome:
        """
        Selenium WebDriver(Chrome) 객체를 생성하여 브라우저를 시작합니다.
        """
        options = Options()
        # 옵션 설정 (예: --no-sandbox, --disable-dev-shm-usage 등)
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
        options.add_argument("--lang=ko_KR")
        #options.add_argument("--charset=utf-8")
        
        if webdriver_path:
            service = Service(webdriver_path)
        else:
            service = Service()  # 시스템 환경 변수에 설정된 경로 사용
        
        self.driver = webdriver.Chrome(service=service, options=options)
        return self.driver


    def check_if_english(self, text):
        """
        주어진 텍스트가 영어인지 판별합니다.
        
        :param text: 판별할 텍스트
        :return: 텍스트가 영어이면 True, 아니면 False
        """
        try:
            language = detect(text)
            return language == "en"
        except Exception as e:
            print("언어 감지 중 오류 발생:", e)
            return False

In [ ]:
def run_crawler():
    # 인자들을 직접 변수로 정의 (필요에 따라 값을 변경하세요)
    num_reviews = 100
    webdriver_path = None  # 시스템 PATH에 크롬드라이버가 있으면 None, 아니면 경로 지정
    json_file = r"database\book_links2.json"  # JSON 파일 경로 (윈도우 환경의 경우 raw string 추천) (page1 #501~)
    start_index = 101
    end_index = 1000

    # BookInfoCrawler 인스턴스 생성
    crawler = BookInfoCrawler(num_reviews, webdriver_path)
    crawler.start_browser()
    
    # MongoDB 연결용 객체 생성 및 DB 연결
    mongo_inserter = MongoDBInserter()
    mongo_inserter.connect_db()
    
    try:
        # JSON 파일에 저장된 책 링크 중 start_index ~ end_index 범위의 도서를 처리
        crawler.scrap_list(
            json_file=json_file,
            start_index=start_index,
            end_index=end_index,
            mongo_inserter=mongo_inserter
        )
        print("전체 수집 완료")
    except Exception as e:
        print("크롤링 중 오류 발생:", e)
    finally:
        crawler.close_browser()

# %% [code]
# run_crawler() 호출하여 크롤러 실행
run_crawler()


MongoDB 연결 성공
책 링크 처리 (Index 101): https://www.goodreads.com/book/show/50714981-corporate-zombies
버튼 클릭 성공!
언어 필터 적용 완료
published 정보 추출 중 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@data-testid='contentContainer' and contains(text(), ' by ')]"}
  (Session info: chrome=133.0.6943.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B55B6EE5+28773]
	(No symbol) [0x00007FF6B55225D0]
	(No symbol) [0x00007FF6B53B8FAA]
	(No symbol) [0x00007FF6B540F286]
	(No symbol) [0x00007FF6B540F4BC]
	(No symbol) [0x00007FF6B5462A27]
	(No symbol) [0x00007FF6B543728F]
	(No symbol) [0x00007FF6B545F6F3]
	(No symbol) [0x00007FF6B5437023]
	(No symbol) [0x00007FF6B53FFF5E]
	(No symbol) [0x00007FF6B54011E3]
	GetHandleVerifier [0x00007FF6B590422D+3490733]
	GetHandleVerifier [0x00007FF6B591BA13+3586963]
	GetHandleVerifier [0x0